In [ ]:
# Import necessary packages

import numpy as np
import opendssdirect as dss
# from utils.device.Inverter import Inverter
# from utils.controller.AdaptiveInvController import AdaptiveInvController
# from utils.controller.FixedInvController import FixedInvController
import matplotlib.pyplot as plt
from math import tan,acos
import copy
import pandas as pd
import time

from collections import deque
import signal_processing as signal_processing

# import numpy as np
# from matplotlib.mlab import PCA
# from sklearn.decomposition import PCA

import scipy.linalg as spla

In [ ]:
# Set global variables and parameters, and test OpenDSS with network


#######################################################
###Global variable initialization and error checking###
#######################################################

# Sbase=1
# LoadScalingFactor = 3
# GenerationScalingFactor = 5

# NoiseMultiplyer= 1
# #Set simulation analysis period - the simulation is from StartTime to EndTime
# StartTime = 40600
# EndTime = StartTime + 640
# EndTime += 1 # creating a list, last element does not count, so we increase EndTime by 1
# #Set hack parameters
# TimeStepOfHack = 160
# PercentHacked = np.array([0,0,0,0,0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0])

# #Set initial VBP parameters for uncompromised inverters
# VBP_normal=np.array([0.97, 1, 1, 1.03, 1.06])
# VBP_attack=np.array([0.997, 1, 1, 1.003, 1.007])

# #Set delays for each node
# Delay_VBPCurveShift = (30+2*np.random.randn(31)).astype(int)
# #Delay_VBPCurveShift = (10+2*np.random.randn(31)).astype(int)
# lpf_meas_vector = (1+0.2*np.random.randn(31))
# lpf_output_vector= (0.1+0.015*np.random.randn(31))

# #Set observer voltage threshold
# ThreshHold_vqvp = 0.06
# adaptive_gain=800


# power_factor=0.9
# pf_converted=tan(acos(power_factor))
# # Number_of_Inverters = 13 #even feeder is 34Bus, we only have 13 inverters




# #Error checking of the global variable -- TODO: add error handling here!
# if EndTime < StartTime or EndTime < 0 or StartTime < 0:
#     print('Setup Simulation Times Inappropriately.')
# if NoiseMultiplyer < 0:
#     print('Setup Noise Multiplyer Correctly.')

# Select OpenDSS file
# dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
# dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
# dss.run_command('Redirect 02node_threephase_unbalanced.dss')
dss.run_command('Redirect 06node_transformer.dss')
# dss.run_command('Redirect 06node_threephase_meshed_unbalanced.dss')
# dss.run_command('Redirect IEEE13Nodeckt.dss')

# Set slack bus (sourcebus) voltage reference in p.u.
SlackBusVoltage = 1.00
dss.Vsources.PU(SlackBusVoltage)

# Solve power flow with OpenDSS file
dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('')

print(dss.Settings.VoltageBases())
print(dss.Vsources.BasekV())

print('')

# print(dss.Solution.Converged())

# Print number of buses, and bus names
print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

# Print number of loads, and load names
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

print('')

# Print different voltage measurements
for k1 in range(len(dss.Circuit.AllBusNames())):
    if dss.Circuit.AllBusNames()[k1] == 'sourcebus':
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        source_voltage = dss.Bus.PuVoltage()
        print(dss.Circuit.AllBusNames()[k1])
        print('BASE - LN')
        print(dss.Bus.kVBase())
        print('VMAGANGLE - LN POLAR')
        print(dss.Bus.VMagAngle())
        print('puVMAGANGLE - LN POLAR')
        print(dss.Bus.puVmagAngle())
        print('VOTLAGES - LN CARTESIAN')
        print(dss.Bus.Voltages())
        print('puVOTLAGES - LN CARTESIAN')
        print(dss.Bus.PuVoltage())
        print('VLL - LL CARTESIAN')
        print(dss.Bus.VLL())
        print('puVLL - LL CARTESIAN')
        print(dss.Bus.puVLL())
        
        tempvoltage = dss.Bus.Voltages()
        
        Vab = tempvoltage[0]
        
print('')
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print(dss.Bus.Nodes())
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    print('')
    
for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
    print(dss.Lines.AllNames()[k1])
    print(dss.Lines.Bus1(), ' - ', dss.Lines.Bus2())
    print(dss.Lines.Length())
    Sk = dss.CktElement.Powers()
    print(Sk)
    # print(dss.CktElement.YPrim())
    print(dss.Lines.Phases())
    YPrim = np.asarray(dss.CktElement.YPrim())
    YPrim = YPrim[0::2] + 1j*YPrim[1::2]
    YPrim = YPrim.reshape(2*dss.Lines.Phases(),2*dss.Lines.Phases())
    print(YPrim)
    YPrim = YPrim[0:dss.Lines.Phases(),0:dss.Lines.Phases()]
    print(YPrim)

    print('')

for k1 in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[k1])
    print(dss.Loads.AllNames()[k1])
    print(dss.CktElement.BusNames())
    print(dss.CktElement.NodeOrder())
    print(dss.CktElement.Powers())
    print('')

for k1 in range(len(dss.Capacitors.AllNames())):
    print(dss.Capacitors.AllNames()[k1])
    dss.Capacitors.Name(dss.Capacitors.AllNames()[k1])
    print(dss.CktElement.BusNames())
    print(dss.CktElement.NodeOrder())
    print(dss.CktElement.Powers())
    print('')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    print(dss.RegControls.TapNumber())
    print('')
    
# nnode = len(dss.Circuit.AllBusNames())
# nline = len(dss.Lines.AllNames())

# print(source_voltage)
# print(nline)


In [ ]:
print(dss.LineCodes.AllNames())

# print(dss.LineCodes.mtx607.Units())

# print(dss.run_command('?LineCode.mtx606.Units'))

for k1 in range(len(dss.LineCodes.AllNames())):
    print(dss.LineCodes.AllNames()[k1])
    dss.LineCodes.Name(dss.LineCodes.AllNames()[k1])
    print(dss.LineCodes.Units())

In [ ]:
for k1 in range(len(dss.Loads.AllNames())):
    print(dss.Loads.AllNames()[k1])
    dss.Loads.Name(dss.Loads.AllNames()[k1])
    print(dss.CktElement.BusNames())
    print(dss.CktElement.NodeOrder())

In [ ]:
import re

for n in range(len(dss.Capacitors.AllNames())):
    print(dss.Capacitors.AllNames()[n])
    dss.Capacitors.Name(dss.Capacitors.AllNames()[n])
    print(dss.CktElement.BusNames()[0])
    #print(dss.CktElement.BusNames()[0])
    pattern =  r"(\w+)\."  #if it is, is the phase present?
    m = re.findall(pattern, dss.CktElement.BusNames()[0])
    print(m)
    # print(pattern)
    print(dss.Capacitors.kvar())

In [ ]:
cap_dict_kvar = {}
cap_dict_kv = {}
for n in range(len(dss.Capacitors.AllNames())):
    dss.Capacitors.Name(dss.Capacitors.AllNames()[n])
    #print(dss.CktElement.BusNames()[0])
    pattern =  r"(\w+)\."  #if it is, is the phase present?
    m = re.findall(pattern, dss.CktElement.BusNames()[0])
    cap_dict_kvar[m[0]] = dss.Capacitors.kvar()
    cap_dict_kv[m[0]] = dss.Capacitors.kV()
    #print(dss.Capacitors.kvar()*1000/Sbase)
    print(cap_dict_kvar)

In [ ]:
for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
    print(dss.Lines.AllNames()[k1])
    print(dss.Lines.Bus1(), ' - ', dss.Lines.Bus2())
    # print(dss.Lines.Length())
    # print(dss.Lines.RMatrix())
    # print(dss.Lines.XMatrix())
    RM = np.asarray(dss.Lines.RMatrix())
    XM = np.asarray(dss.Lines.XMatrix())
    ZM = RM + 1j*XM
    ZM = np.reshape(ZM,(int(ZM.shape[0]**(1/2)),int(ZM.shape[0]**(1/2))))
    ZM = ZM*dss.Lines.Length()

    print('ZMatrix - nonpadded')
    print(ZM)
    print('Inverse ZMatrix - nonpadded')
    print(spla.inv(ZM))
    # Z = np.array((dss.Lines.RMatrix().shape),dtype='complex')
    # print(dss.CktElement.YPrim())
    YP = np.asarray(dss.CktElement.YPrim(),dtype='complex')
    # print(YP)
    YP = YP[0:-1:2] + 1j*YP[1::2]
    # print(YP)
    # print(YP.shape[0]**(1/2))
    YP = np.reshape(YP,(int(YP.shape[0]**(1/2)),int(YP.shape[0]**(1/2))))
    print('Ymatrix - nonpadded')
    print(YP)
        
    YP11 = YP[0:int(YP.shape[0]/2),0:int(YP.shape[0]/2)]
    print('YP11')
    print(YP11)
    print(ZM@YP11)

    YP12 = YP[0:int(YP.shape[0]/2),int(YP.shape[0]/2):]
    print('YP12')
    print(YP12)
    print(ZM@YP12)

    YP21 = YP[int(YP.shape[0]/2):,0:int(YP.shape[0]/2)]
    print('YP21')
    print(YP21)
    print(ZM@YP21)

    YP22 = YP[int(YP.shape[0]/2):,int(YP.shape[0]/2):]
    print('YP22')
    print(YP22)
    print(ZM@YP22)

    print('')

In [ ]:
for k1 in range(len(dss.Transformers.AllNames())):
    dss.Transformers.Name(dss.Transformers.AllNames()[k1])
    print(dss.Transformers.AllNames()[k1])
    print(dss.CktElement.Voltages())
    print(dss.CktElement.Currents())
    print(dss.CktElement.Powers())

In [ ]:
# Set global variables and parameters, and test OpenDSS with network


#######################################################
###Global variable initialization and error checking###
#######################################################

# Sbase=1
# LoadScalingFactor = 3
# GenerationScalingFactor = 5

# NoiseMultiplyer= 1
# #Set simulation analysis period - the simulation is from StartTime to EndTime
# StartTime = 40600
# EndTime = StartTime + 640
# EndTime += 1 # creating a list, last element does not count, so we increase EndTime by 1
# #Set hack parameters
# TimeStepOfHack = 160
# PercentHacked = np.array([0,0,0,0,0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0])

# #Set initial VBP parameters for uncompromised inverters
# VBP_normal=np.array([0.97, 1, 1, 1.03, 1.06])
# VBP_attack=np.array([0.997, 1, 1, 1.003, 1.007])

# #Set delays for each node
# Delay_VBPCurveShift = (30+2*np.random.randn(31)).astype(int)
# #Delay_VBPCurveShift = (10+2*np.random.randn(31)).astype(int)
# lpf_meas_vector = (1+0.2*np.random.randn(31))
# lpf_output_vector= (0.1+0.015*np.random.randn(31))

# #Set observer voltage threshold
# ThreshHold_vqvp = 0.06
# adaptive_gain=800


# power_factor=0.9
# pf_converted=tan(acos(power_factor))
# # Number_of_Inverters = 13 #even feeder is 34Bus, we only have 13 inverters




# #Error checking of the global variable -- TODO: add error handling here!
# if EndTime < StartTime or EndTime < 0 or StartTime < 0:
#     print('Setup Simulation Times Inappropriately.')
# if NoiseMultiplyer < 0:
#     print('Setup Noise Multiplyer Correctly.')

# Select OpenDSS file
# dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
# dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
# dss.run_command('Redirect 02node_threephase_unbalanced.dss')
dss.run_command('Redirect 06node_threephase_radial_unbalanced_regulator.dss')
dss.run_command('Redirect 06node_transformer.dss')
# dss.run_command('Redirect 06node_threephase_meshed_unbalanced.dss')
# dss.run_command('Redirect IEEE13Nodeckt.dss')

# Set slack bus (sourcebus) voltage reference in p.u.
SlackBusVoltage = 1.00
dss.Vsources.PU(SlackBusVoltage)

# Solve power flow with OpenDSS file
dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('')

print(dss.Settings.VoltageBases())
print(dss.Vsources.BasekV())

print('')

# print(dss.Solution.Converged())

# Print number of buses, and bus names
print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

# Print number of loads, and load names
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    print(dss.RegControls.TapNumber())
    dss.RegControls.TapNumber(8)
    print(dss.RegControls.TapNumber())
    print('')

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')

print('')

# Print different voltage measurements
for k1 in range(len(dss.Circuit.AllBusNames())):
    if dss.Circuit.AllBusNames()[k1] == 'sourcebus':
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        source_voltage = dss.Bus.PuVoltage()
        print(dss.Circuit.AllBusNames()[k1])
        print('BASE - LN')
        print(dss.Bus.kVBase())
        print('VMAGANGLE - LN POLAR')
        print(dss.Bus.VMagAngle())
        print('puVMAGANGLE - LN POLAR')
        print(dss.Bus.puVmagAngle())
        print('VOTLAGES - LN CARTESIAN')
        print(dss.Bus.Voltages())
        print('puVOTLAGES - LN CARTESIAN')
        print(dss.Bus.PuVoltage())
        print('VLL - LL CARTESIAN')
        print(dss.Bus.VLL())
        print('puVLL - LL CARTESIAN')
        print(dss.Bus.puVLL())
        
        tempvoltage = dss.Bus.Voltages()
        
        Vab = tempvoltage[0]
        
print('')
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print(dss.Bus.Nodes())
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    print('')
    
for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
    print(dss.Lines.AllNames()[k1])
    print(dss.Lines.Bus1(), ' - ', dss.Lines.Bus2())
    print(dss.Lines.Length())
    Sk = dss.CktElement.Powers()
    print(Sk)
    # print(dss.CktElement.YPrim())
    print(dss.Lines.Phases())
    YPrim = np.asarray(dss.CktElement.YPrim())
    YPrim = YPrim[0::2] + 1j*YPrim[1::2]
    YPrim = YPrim.reshape(2*dss.Lines.Phases(),2*dss.Lines.Phases())
    print(YPrim)
    YPrim = YPrim[0:dss.Lines.Phases(),0:dss.Lines.Phases()]
    print(YPrim)

    print('')

for k1 in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[k1])
    print(dss.Loads.AllNames()[k1])
    print(dss.CktElement.BusNames())
    print(dss.CktElement.NodeOrder())
    print(dss.CktElement.Powers())
    print('')

for k1 in range(len(dss.Capacitors.AllNames())):
    print(dss.Capacitors.AllNames()[k1])
    dss.Capacitors.Name(dss.Capacitors.AllNames()[k1])
    print(dss.CktElement.BusNames())
    print(dss.CktElement.NodeOrder())
    print(dss.CktElement.Powers())
    print('')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    print(dss.RegControls.TapNumber())
    print('')

bus1 = 'a01'
bus2 = 'a02'

dss.Circuit.SetActiveBus(bus1)
V1 = np.asarray(dss.Bus.PuVoltage())
print(V1)

dss.Circuit.SetActiveBus(bus2)
V2 = np.asarray(dss.Bus.PuVoltage())
print(V2)

print(V2/V1)
    
# nnode = len(dss.Circuit.AllBusNames())
# nline = len(dss.Lines.AllNames())

# print(source_voltage)
# print(nline)

VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')
nnode = len(dss.Circuit.AllBusNames())
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
    print(ph)
    Vtemp = np.asarray(dss.Bus.PuVoltage())
    Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
    VDSS[ph,k1] = Vtemp    
print('VDSS\n', np.round(VDSS,decimals=6))


In [ ]:
VDSS

In [ ]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys
from lib.compute_KCL_matrices import compute_KCL_matrices
from lib.basematrices import basematrices
from lib.NR3_timevarying import NR3_timevarying
from lib.network_mapper_OpenDSSnonvec import network_mapper_function as nmf2
from lib.NR3 import NR3_function
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

#fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = '06node_threephase_unbalanced.dss'
fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
fn = '06node_transformer.dss'

# Time-steps
times = np.linspace(0, 2*np.pi, 5)
times = [0]
# DER changes over time-steps
der = [0, 0, 0, 0, 0]
# Capacitance changes over time-steps
capacitance = [0, 0, 0, 0, 0]

dss.run_command('Redirect ' + fn)
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

VNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR01=  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")


# Pre-generate matrices
#X, g_SB, b_SB, G_KVL, b_KVL, H, g, b = basematrices(fn, slackidx, Vslack, None, None)
X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, H_reg, G_reg = basematrices(fn, slackidx, Vslack, None, None)

for i in range(len(times)):
    # Run NR3 with variations in time, DER, and capacitance
   # VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
    VNR=    NR3_timevarying(fn, X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, None, None, der[i], capacitance[i], times[i], H_reg, G_reg )    
#     VNR01[i, :, :] = np.reshape(VNR, (3, nnode))
#     INR01[i, :, :] = np.reshape(INR, (3, nline))
#     STXNR01[i, :, :] = np.reshape(STXNR, (3, nline))
#     SRXNR01[i, :, :] = np.reshape(SRXNR, (3, nline))
#     iNR01[i, :, :] = np.reshape(iNR, (3, nnode))
#     sNR01[i, :, :] = np.reshape(sNR, (3, nnode))


In [ ]:
print(VDSS)
print(VNR)

In [ ]:
print(np.max(np.abs(VDSS-VNR)))